# Capstone Project - The Battle of the Neighborhoods

#### Project Instructions : In a city of your choice, if someone is looking to open a restaurant, where would you recommend that they open it? No matter what you decide to do, make sure to provide sufficient justification of why you think what you want to do or solve is important and why would a client or a group of people be interested in your project.

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)


## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal neighborhood for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening a restaurant in **Toronto**, Canada.

Since there are lots of restaurants in Toronto we will try to detect **neighborhoods that have - various types of restaurants as their most common venue**. By doing so we'll ensure that there is a demand for a restaurants in that neighborhood and our client can safely open one. 

We will use our data science powers to generate the most promising neighborhood based on this criterion. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.


## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* neighborhoods that have many restaurants as their most common venues

Following data sources will be needed to extract/generate the required information:
* Get the Borough and neighborhood information from Wikipedia 
* As Google Maps API is taking a long time to load and is failing hence we are using the geospatial data provided - http://cocl.us/Geospatial_data to perform the task of updating the coordinates and create clusters
* The number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* We'll use Folium package to create the map of Toronto using latitude and longitude values

## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting neighborhoods of Toronto that have high restaurant density, particularly those with which have restaurants as their most common venues.

In Part 1, We'll gather Toronto's neighborhood information from Wikipedia sources and created a clean dataframe.

In Part 2, We'll prepare the data frames with Neighborhood coordinates, postal code, borough and neighborhood information that will be used for clustering. We also used the geospatial data from the link. 

In the final Part 3, we will focus on most promising areas and within those create **clusters of locations that meet our requirements** established in discussion with stakeholders: we will take into consideration locations with **multiple restaurants in the vicinity**, and we want locations **that have various restaurants as their most common venue**. We will present map of all such locations but also create clusters (using **k-means clustering**) of those locations to identify general zones / neighborhoods where the conditions for opening a new restaurant will be the best.

### Part 1 - Gathering Toronto's neighborhood information from sources and creating a clean dataframe

In [3]:
# importing libraries
import pandas as pd

In [15]:
# Get the Borough and neighborhood information from Wikipedia 
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
newdata= pd.read_html(url,header=0)
newdata[0].head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [16]:
df = newdata[0]

#### Dataframe processing - updating the column names

In [27]:
df.columns = ['PostalCode', 'Borough', 'Neighborhood']
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
# Saving the dataframe as a file for reference 
df.to_csv(r'C:\Users\subha\Desktop\Final Project - The Battle of Neighborhoods\df.csv')

#### Deleting the new lines from the data frame

In [28]:
# Deleting the new lines from the data frame
df = df.replace('\n',' ', regex=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Dataframe processing - deleting the rows with Borough = 'Not assigned'

In [29]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)

# Reset the index and drop the previous index
df = df.reset_index(drop=True)

df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [30]:
#Combine the neighborhoods with same PostalCode and Boroughs
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(','.join).reset_index()
df.columns = ['PostalCode','Borough','Neighborhood']
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Remove spaces at the beginning of the strings

In [31]:
df['Neighborhood'] = df['Neighborhood'].str.strip()

#### Assigning Borough values to the Neignbourhood where vlaue is "Not assigned"

In [32]:
df.loc[df['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df['Borough']

#### Check the dataset to see if the Neighbourhood for Queen's Park has been updated with comma separator

In [33]:
df[df['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Harbourfront,Regent Park"


In [18]:
# Save the file as csv for reference
df.to_csv(r'C:\Users\subha\Desktop\Final Project - The Battle of Neighborhoods\df_after_part1.csv')

### Part 2 - Preparing the data frames with Neighborhood coordinates, postal code, borough and neighborhood information

#### I used the csv data provided (http://cocl.us/Geospatial_data) to perform the task

In [34]:
import numpy as np 

link = "http://cocl.us/Geospatial_data"
df1 = pd.read_csv(link)

df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [35]:
df1.shape

(103, 3)

#### Updating the column name - "Postal code" to "PostalCode" as I used in my dataframe to combine the two data frames together

In [36]:
df1.columns = ['PostalCode','Latitude','Longitude']
df1.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Reading the csv file I saved in the Part 1 of the project : df_after_part1.csv

In [20]:
df = pd.read_csv('https://raw.githubusercontent.com/Subhayans/Coursera_Capstone/master/df_after_part1.csv', index_col=0)

In [37]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Merging the two data frames - df & df1 together based on their PostalCode

In [38]:
df_merged = pd.merge(df, df1, on='PostalCode')
df_merged.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Save the dataframe as a csv file for reference

In [25]:
df_merged.to_csv(r'C:\Users\subha\Desktop\Final Project - The Battle of Neighborhoods\df_merged.csv')

### Part 3 - Segmenting and Clustering Neighborhoods in Toronto and identrifying the best neighborhoods for restaurant business

In [39]:
# Importng the required libraries 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 
from geopy.geocoders import Nominatim # converts an address into latitude and longitude values

import requests # this is a library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Import Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means for clustering part of the assignment
from sklearn.cluster import KMeans

import folium # folium is a map rendering library

print('Libraries successfully imported...')


Libraries successfully imported...


In [40]:
link = "https://raw.githubusercontent.com/Subhayans/Coursera_Capstone/master/df_merged.csv"
df_complete = pd.read_csv(link,index_col=0)

df_complete.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Find out the number of boroughs and neighborhoods

In [41]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_complete['Borough'].unique()),
        df_complete.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


#### We used geopy library to get the latitude and longitude values of Toronto

In [42]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="can_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto to visualize the neighbourhoods

In [43]:
# Create a map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_complete['Latitude'], df_complete['Longitude'], df_complete['Borough'], df_complete['Neighborhood']):
    label = '{}, {}'.format(df_complete, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Next I utilized the Foursquare API to explore the neighborhoods and group or segment them

In [44]:
# Define Foursquare Credentials and Version
LIMIT = 50

CLIENT_ID = '3LR4K4KMMWMKHVB43OMYLJX3MYH2OJPJUSHKCPK13VFKS2XX' # My Foursquare ID
CLIENT_SECRET = 'W2EJI4BUB1URZQGUCU55RP4TVJAPZSJ2TXJMUZSBTDPECVMW' # My Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3LR4K4KMMWMKHVB43OMYLJX3MYH2OJPJUSHKCPK13VFKS2XX
CLIENT_SECRET:W2EJI4BUB1URZQGUCU55RP4TVJAPZSJ2TXJMUZSBTDPECVMW


#### Explore Neighborhoods in Toronto
#### Create a function to repeat the same process to all the neighborhoods in Toronto

In [45]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now we write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues_df

In [46]:
toronto_venues_df = getNearbyVenues(names=df_complete['Neighborhood'],
                                   latitudes=df_complete['Latitude'],
                                   longitudes=df_complete['Longitude'],
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

#### Glimspe of the new data set

In [47]:
toronto_venues_df.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [48]:
#Check the size of the dataframe
toronto_venues_df.shape

(1701, 7)

#### Let's check how many venues were returned for each neighborhood

In [49]:
toronto_venues_df.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",50,50,50,50,50,50
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",11,11,11,11,11,11
"Alderwood,Long Branch",8,8,8,8,8,8
"Bathurst Manor,Downsview North,Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",24,24,24,24,24,24
Berczy Park,50,50,50,50,50,50


#### Let's find out how many unique categories can be curated from all the returned venues

In [50]:
print('There are {} uniques categories.'.format(len(toronto_venues_df['Venue Category'].unique())))

There are 255 uniques categories.


#### Next let's analyze each Neighborhood

In [51]:
# Doing one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues_df[['Venue Category']], prefix="", prefix_sep="")

# Adding neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues_df['Neighborhood'] 

# Moving neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Rou

In [52]:
# Check the new data frame size
toronto_onehot.shape

(1701, 255)

In [52]:
# Saving the dataframe as a file for reference 
toronto_onehot.to_csv(r'C:\Users\subha\Desktop\Final Project - The Battle of Neighborhoods\toronto_onehot.csv')

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [53]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.00,0.00,0.00,0.00,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.020000,0.000000,0.040000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.02,0.040000,0.000000,0.000000,0.000000,0.0,0.020000,0.020000,

In [55]:
# Saving the dataframe as a file for reference 
toronto_grouped.to_csv(r'C:\Users\subha\Desktop\Final Project - The Battle of Neighborhoods\toronto_grouped.csv')

In [54]:
# confirm the new size
toronto_grouped.shape

(100, 255)

#### Printing each neighborhood along with the top 5 most common venues

In [55]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
            venue  freq
0     Coffee Shop  0.08
1      Steakhouse  0.06
2            Café  0.06
3       Gastropub  0.04
4  Breakfast Spot  0.04


----Agincourt----
                venue  freq
0              Lounge  0.25
1      Breakfast Spot  0.25
2        Skating Rink  0.25
3      Sandwich Place  0.25
4  Mac & Cheese Joint  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                      venue  freq
0                Playground   0.5
1                      Park   0.5
2               Yoga Studio   0.0
3  Mediterranean Restaurant   0.0
4        Miscellaneous Shop   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                 venue  freq
0        Grocery Store  0.18
1             Pharmacy  0.09
2          Coffee Shop  0.09
3  Japanese Restaurant  0.09
4       Discount Store  0.09


----Alderwood,Long Branch----
          venue  freq
0   Pizza Place  0.25
1     

              venue  freq
0  Department Store  0.17
1       Bus Station  0.17
2       Coffee Shop  0.17
3        Hobby Shop  0.17
4    Discount Store  0.17


----East Toronto----
                        venue  freq
0                        Park  0.33
1           Convenience Store  0.33
2               Metro Station  0.33
3                 Yoga Studio  0.00
4  Modern European Restaurant  0.00


----Emery,Humberlea----
                        venue  freq
0      Furniture / Home Store  0.33
1  Construction & Landscaping  0.33
2              Baseball Field  0.33
3                 Men's Store  0.00
4           Mobile Phone Shop  0.00


----Fairview,Henry Farm,Oriole----
                  venue  freq
0        Clothing Store  0.16
1           Coffee Shop  0.10
2  Fast Food Restaurant  0.08
3         Women's Store  0.04
4      Toy / Game Store  0.04


----First Canadian Place,Underground city----
          venue  freq
0          Café  0.12
1   Coffee Shop  0.08
2         Hotel  0.06
3    Resta

                        venue  freq
0               Women's Store   0.5
1                  Playground   0.5
2  Modern European Restaurant   0.0
3          Mac & Cheese Joint   0.0
4            Malay Restaurant   0.0


----St. James Town----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.06
2    Gastropub  0.06
3        Hotel  0.06
4   Restaurant  0.06


----Stn A PO Boxes 25 The Esplanade----
                venue  freq
0                Café  0.08
1        Cocktail Bar  0.06
2         Coffee Shop  0.06
3              Bakery  0.04
4  Seafood Restaurant  0.04


----Studio District----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2  American Restaurant  0.05
3               Bakery  0.05
4   Italian Restaurant  0.05


----The Annex,North Midtown,Yorkville----
            venue  freq
0     Coffee Shop  0.12
1  Sandwich Place  0.12
2            Café  0.12
3     Pizza Place  0.08
4    Burger Joint  0.04


----The Beaches----
             

#### Let's put that into a pandas dataframe
#### 1st, let's write a function to sort the venues in descending order.

In [56]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 venues for each neighborhood

In [57]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Hotel,Bar,Gastropub,Breakfast Spot,American Restaurant,Asian Restaurant,Pizza Place
1,Agincourt,Lounge,Skating Rink,Breakfast Spot,Sandwich Place,Women's Store,Electronics Store,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Curling Ice,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run,Discount Store
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Beer Store,Fried Chicken Joint,Sandwich Place,Japanese Restaurant,Fast Food Restaurant,Discount Store,Pizza Place,Pharmacy,Coffee Shop
4,"Alderwood,Long Branch",Pizza Place,Gym,Skating Rink,Sandwich Place,Pub,Coffee Shop,Pharmacy,College Stadium,Drugstore,College Auditorium


In [61]:
# Save the dataframe as a csv for reference
neighborhoods_venues_sorted.to_csv(r'C:\Users\subha\Desktop\Final Project - The Battle of Neighborhoods\neighborhoods_venues_sorted.csv')

#### Cluster Neighborhoods
#### Run k-means to cluster the neighborhood into 5 clusters.

In [58]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 0, 2, 2, 1, 1, 1, 1, 1])

#### Next we create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [59]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_complete

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,2.0,Fast Food Restaurant,Women's Store,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,4.0,Bar,Moving Target,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,1.0,Electronics Store,Mexican Restaurant,Breakfast Spot,Intersection,Medical Center,Rental Car Location,Pizza Place,Deli / Bodega,Drugstore,Donut Shop
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Convenience Store,Korean Restaurant,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Fried Chicken Joint,Bakery,Hakka Restaurant,Bank,Athletics & Sports,Thai Restaurant,Caribbean Restaurant,Diner,Dessert Shop,Dim Sum Restaurant


In [60]:
#Check the added cluster labels
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Hotel,Bar,Gastropub,Breakfast Spot,American Restaurant,Asian Restaurant,Pizza Place
1,1,Agincourt,Lounge,Skating Rink,Breakfast Spot,Sandwich Place,Women's Store,Electronics Store,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run
2,0,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Curling Ice,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run,Discount Store
3,2,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Beer Store,Fried Chicken Joint,Sandwich Place,Japanese Restaurant,Fast Food Restaurant,Discount Store,Pizza Place,Pharmacy,Coffee Shop
4,2,"Alderwood,Long Branch",Pizza Place,Gym,Skating Rink,Sandwich Place,Pub,Coffee Shop,Pharmacy,College Stadium,Drugstore,College Auditorium


#### Finally, let's visualize the resulting clusters

In [61]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine the Clusters: 
#### Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we assigned a name to each cluster.

#### Cluster 1 - Segmentation based on Parks and Playgrounds

In [62]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,0.0,Park,Playground,Curling Ice,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run,Discount Store
21,North York,0.0,Park,Women's Store,Curling Ice,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run,Discount Store
23,North York,0.0,Park,Bank,Convenience Store,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run
25,North York,0.0,Bus Stop,Park,Food & Drink Shop,Women's Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run,Discount Store
30,North York,0.0,Park,Airport,Women's Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop
40,East York,0.0,Park,Metro Station,Convenience Store,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run
44,Central Toronto,0.0,Park,Swim School,Bus Line,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run
48,Central Toronto,0.0,Park,Playground,Restaurant,College Cafeteria,Cupcake Shop,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run,Discount Store
50,Downtown Toronto,0.0,Park,Trail,Playground,Building,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
74,York,0.0,Park,Women's Store,Fast Food Restaurant,Market,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run


### Cluster 2 - Segmentation based on Miscellaneous stores, restaurants, cafes etc

In [63]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,1.0,Electronics Store,Mexican Restaurant,Breakfast Spot,Intersection,Medical Center,Rental Car Location,Pizza Place,Deli / Bodega,Drugstore,Donut Shop
3,Scarborough,1.0,Coffee Shop,Convenience Store,Korean Restaurant,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop
4,Scarborough,1.0,Fried Chicken Joint,Bakery,Hakka Restaurant,Bank,Athletics & Sports,Thai Restaurant,Caribbean Restaurant,Diner,Dessert Shop,Dim Sum Restaurant
5,Scarborough,1.0,Women's Store,Playground,Curling Ice,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run,Discount Store
6,Scarborough,1.0,Bus Station,Hobby Shop,Coffee Shop,Department Store,Discount Store,Convenience Store,Dessert Shop,Dim Sum Restaurant,Diner,Women's Store
7,Scarborough,1.0,Bus Line,Bakery,Park,Soccer Field,Fast Food Restaurant,Metro Station,Intersection,Women's Store,Diner,Dessert Shop
8,Scarborough,1.0,American Restaurant,Motel,Movie Theater,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Women's Store,Dance Studio
9,Scarborough,1.0,Café,College Stadium,Skating Rink,General Entertainment,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
10,Scarborough,1.0,Indian Restaurant,Latin American Restaurant,Pet Store,Vietnamese Restaurant,Chinese Restaurant,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant
11,Scarborough,1.0,Middle Eastern Restaurant,Auto Garage,Shopping Mall,Breakfast Spot,Sandwich Place,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run,Discount Store


### Cluster 3 - Segmentation based on Restaurants, Pizza Places

In [64]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,2.0,Fast Food Restaurant,Women's Store,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run
13,Scarborough,2.0,Pizza Place,Chinese Restaurant,Italian Restaurant,Noodle House,Bank,Breakfast Spot,Fried Chicken Joint,Fast Food Restaurant,Thai Restaurant,Dessert Shop
15,Scarborough,2.0,Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Pizza Place,Sandwich Place,Bubble Tea Shop,Thrift / Vintage Store,Grocery Store,Pharmacy,Breakfast Spot
17,North York,2.0,Dog Run,Golf Course,Fast Food Restaurant,Mediterranean Restaurant,Pool,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
24,North York,2.0,Coffee Shop,Butcher,Home Service,Discount Store,Pharmacy,Pizza Place,Grocery Store,Airport Terminal,Falafel Restaurant,Ethiopian Restaurant
34,North York,2.0,Coffee Shop,Portuguese Restaurant,Pizza Place,Hockey Arena,Intersection,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
35,East York,2.0,Pizza Place,Fast Food Restaurant,Bank,Gym / Fitness Center,Pet Store,Pharmacy,Gastropub,Intersection,Athletics & Sports,Dessert Shop
81,York,2.0,Bus Line,Pizza Place,Caribbean Restaurant,Women's Store,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Discount Store
89,Etobicoke,2.0,Pizza Place,Gym,Skating Rink,Sandwich Place,Pub,Coffee Shop,Pharmacy,College Stadium,Drugstore,College Auditorium
96,North York,2.0,Empanada Restaurant,Pizza Place,Curling Ice,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run,Discount Store


### Cluster 4 

In [65]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,Etobicoke,3.0,Bank,Women's Store,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run


### Cluster 5

In [66]:

toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,4.0,Bar,Moving Target,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run


## Results and Discussion <a name="results"></a>

Our analysis shows that although there is a great number of restaurants in Toronto, yet the "1st Most Common Venue" for neighborhoods changes from Parks to Bus Stops to coffee shops to various stores and restaurants. Highest concentration of restaurants was detected in Scarborough and North York.

From the segmentations in Cluster 2, Cluster 3 above, it is evident that Scarborough is the best place to open restaurants - as it has a wide variety of restaurants as the restaurants in this neighborhood accounts for the maximum numeber of "1st Most Common Venue". The various types of restaurants include -  American Restaurant, Indian Restaurant, Greek Restaurant, Middle-Eastern Restaurant, Fried Chicken Joint, Chinese Restaurant, Fast Food Restaurant, Pizza Place etc. Which means our client can choose both to open a new restaurant that is unique to the area or a similar restaurant like the ones already existing.  

## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify Toronto neighborhoods in order to aid the stakeholder in narrowing down the search for optimal location for a new restaurant. 

We first created the boroughs and neighborhoods from various data sources and added their location coordinates using geocode. 
Then we created a map to visualize Toronto's neighborhoods. Next we utilized the Foursquare API to explore all the neighborhoods and their most frequented venues. Then we analyzed each Neighborhood by taking the mean of the frequency of venue-occurrence for each category. Then we created a dataframe with the top 10 - most common venues in each neighborhood. 
Then we used k-means clustering of those locations in order to create major neighborhoods of interest hat were to be used as starting points for final exploration by stakeholders.

Final decission on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in the recommended neighborhood (Scarborough), taking into consideration additional factors like attractiveness of restaurant location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every location etc.